<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/Liver_Ultrasound_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 2 Hackathon - Liver_Ultrasound_Detection

## Mounting Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Explore Datasets

### Load Datasets

In [ ]:
!unzip /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/zips/liver-ultrasound-detection.zip

### Explore Train Images Directory

In [15]:
import os

In [16]:
n_images = len(os.listdir('/content/train/train/images'))
n_lesions = len(os.listdir('/content/train/train/annotations'))
n_non_lesions = n_images - n_lesions

print('num of datas =' , n_images , 'num of lesion =', n_lesions , 'num of non lesion' , n_non_lesions)

num of datas = 14448 num of lesion = 7222 num of non lesion 7226


### Explore Train Annotations Directory

In [17]:
annotations_path = '/content/train/train/annotations'

with open (f'{annotations_path}/{100022}.txt' , 'r') as file :

    file_content = file.read().split(' ')

    print(f'class = {file_content[0]}\ncenter_x = {file_content[1]}\ncenter_y = {file_content[2]}\nbox_width / image_width = {file_content[3]}\nbox_height / image_height = {file_content[4]}')

class = 2
center_x = 0.4601851851851852
center_y = 0.30185185185185187
box_width / image_width = 0.08703703703703704
box_height / image_height = 0.12222222222222222



## Setup Format Datasets For YOLOv10

### Define Path Of Directories

In [18]:
import os
import pandas as pd

In [19]:
train_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/train/images'
validation_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/validation/images'
test_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/test/images'

In [20]:
train_labels_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/train/labels'
validation_labels_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/validation/labels'

### Managing With Annotation Mapping File

In [21]:
annotation_mapping_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_mapping.csv'
annotation_mapping = pd.read_csv(annotation_mapping_path)

In [22]:
annotation_mapping

,Image File,Annotation File,Source,Type
0,24084.jpg,24084.txt,machine,machine_positive
1,97528.jpg,97528.txt,machine,machine_positive
2,54563.jpg,54563.txt,machine,machine_positive
3,115211.jpg,115211.txt,machine,machine_positive
4,71913.jpg,71913.txt,machine,machine_positive
...,...,...,...,...
19341,1160.jpg,NaN,mobile,mobile_negative
19342,153.jpg,NaN,mobile,mobile_negative
19343,130705.jpg,NaN,mobile,mobile_negative
19344,812.jpg,NaN,mobile,mobile_negative


In [23]:
annotation_mapping_trainable = annotation_mapping[annotation_mapping['Annotation File'].isnull() == False]

In [24]:
annotation_mapping_trainable

,Image File,Annotation File,Source,Type
0,24084.jpg,24084.txt,machine,machine_positive
1,97528.jpg,97528.txt,machine,machine_positive
2,54563.jpg,54563.txt,machine,machine_positive
3,115211.jpg,115211.txt,machine,machine_positive
4,71913.jpg,71913.txt,machine,machine_positive
...,...,...,...,...
18662,31309.jpg,31309.txt,mobile,mobile_positive
18663,103330.jpg,103330.txt,mobile,mobile_positive
18664,141103.jpg,141103.txt,mobile,mobile_positive
18665,9070.jpg,9070.txt,mobile,mobile_positive


In [25]:
print(f'num of all files = {len(annotation_mapping)} num of trainable files = {len(annotation_mapping_trainable)}')

num of all files = 19346 num of trainable files = 9667


### Create annotation_for_datasets.csv file

In [26]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm

In [27]:
annotation_for_train_datasets = {

    'file_name' : [] ,
    'width' : [] ,
    'height' : [] ,
    'class' : [] ,

}
#class 0 -> no lesion 1 -> lesion

In [28]:
# loop in train
train_dataset_path = '/content/train/train'

train_dataset_annotations = os.listdir(f'{train_dataset_path}/annotations')

for idx , file_name in tqdm(enumerate(os.listdir(f'{train_dataset_path}/images'))) :

    name = file_name.split('.')[0]

    annotation_for_train_datasets['file_name'].append(name)

    class_ = 1 if f'{name}.txt' in train_dataset_annotations else 0
    annotation_for_train_datasets['class'].append(class_)

    image = Image.open(f'{train_dataset_path}/images/{file_name}')

    shape = np.array(image).shape

    annotation_for_train_datasets['width'].append(shape[0])
    annotation_for_train_datasets['height'].append(shape[1])

14448it [02:45, 87.04it/s]


In [29]:
annotation_for_train_datasets_df = pd.DataFrame.from_dict(annotation_for_train_datasets)
annotation_for_train_datasets_df.set_index('file_name', inplace=True)

In [30]:
annotation_for_train_datasets_df

,width,height,class
file_name,,,
98356,782,1042,1
74796,810,1080,0
121413,810,1080,1
148426,540,720,1
122997,540,720,0
...,...,...,...
69995,540,720,1
116763,810,1080,0
106056,810,1080,1


In [31]:
annotation_for_train_datasets_df.to_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_train_datasets.csv')

In [32]:
annotation_for_train_datasets_df = pd.read_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_train_datasets.csv' , index_col = 'file_name')
annotation_for_train_datasets_df

,width,height,class
file_name,,,
98356,782,1042,1
74796,810,1080,0
121413,810,1080,1
148426,540,720,1
122997,540,720,0
...,...,...,...
69995,540,720,1
116763,810,1080,0
106056,810,1080,1


In [33]:
annotation_for_validation_datasets = {

    'file_name' : [] ,
    'width' : [] ,
    'height' : [] ,
    'class' : [] ,

}
#class 0 -> no lesion 1 -> lesion

In [34]:
# loop in validation
validation_dataset_path = '/content/val/val'

validation_dataset_annotations = os.listdir(f'{validation_dataset_path}/annotations')

for idx , file_name in tqdm(enumerate(os.listdir(f'{validation_dataset_path}/images'))) :

    name = file_name.split('.')[0]

    annotation_for_validation_datasets['file_name'].append(name)
    class_ = 1 if f'{name}.txt' in validation_dataset_annotations else 0

    annotation_for_validation_datasets['class'].append(class_)

    image = Image.open(f'{validation_dataset_path}/images/{file_name}')

    shape = np.array(image).shape

    annotation_for_validation_datasets['width'].append(shape[0])
    annotation_for_validation_datasets['height'].append(shape[1])

4898it [02:44, 29.82it/s]


In [35]:
annotation_for_validation_datasets_df = pd.DataFrame.from_dict(annotation_for_validation_datasets)
annotation_for_validation_datasets_df.set_index('file_name', inplace=True)

In [36]:
annotation_for_validation_datasets_df

,width,height,class
file_name,,,
27908,540,720,0
77411,540,720,0
48899,540,720,1
23485,810,1080,1
129834,614,816,0
...,...,...,...
86856,3024,4032,1
1684,3024,4032,0
10519,660,880,0


In [37]:
annotation_for_validation_datasets_df.to_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_validation_datasets.csv')

In [38]:
annotation_for_validation_datasets_df = pd.read_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_validation_datasets.csv' , index_col = 'file_name')
annotation_for_validation_datasets_df

,width,height,class
file_name,,,
27908,540,720,0
77411,540,720,0
48899,540,720,1
23485,810,1080,1
129834,614,816,0
...,...,...,...
86856,3024,4032,1
1684,3024,4032,0
10519,660,880,0


### Create Images / Labels Folder

In [39]:
import cv2
import os
from tqdm import tqdm

In [40]:
train_dataset_images_path = '/content/train/train/images'
validation_dataset_images_path = '/content/val/val/images'
test_dataset_images_path = '/content/test/test/images'

In [41]:
new_train_dataset_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/train/images'
new_validation_dataset_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/validation/images'
new_test_dataset_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/test/images'

In [70]:
for file_name in tqdm(os.listdir(train_dataset_images_path)) :

    image = cv2.imread(f'{train_dataset_images_path}/{file_name}')
    cv2.imwrite(f'{new_train_dataset_images_path}/{file_name}' , image)

100%|██████████| 14448/14448 [11:17<00:00, 21.32it/s]


In [71]:
for file_name in tqdm(os.listdir(validation_dataset_images_path)) :

    image = cv2.imread(f'{validation_dataset_images_path}/{file_name}')
    cv2.imwrite(f'{new_validation_dataset_images_path}/{file_name}' , image)

100%|██████████| 4898/4898 [12:36<00:00,  6.47it/s]


In [72]:
for file_name in tqdm(os.listdir(test_dataset_images_path)) :

    image = cv2.imread(f'{test_dataset_images_path}/{file_name}')
    cv2.imwrite(f'{new_test_dataset_images_path}/{file_name}' , image)

100%|██████████| 5153/5153 [33:36<00:00,  2.55it/s]


## Dinov2 For Classification

### Define Custom Dataset To lesion / non-lesion Classification

In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
import pandas as pd
import numpy as np

In [2]:
class DatasetForClassification(Dataset):

    def __init__(self, annotation_path , image_path, transform = None , target_transform = None):

        self.annotation = pd.read_csv(annotation_path)
        self.image_path = image_path
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):

        return len(os.listdir(self.image_path))

    def __getitem__(self, idx):

        file_name = self.annotation.iloc[idx]['file_name']

        image = Image.open(f"{self.image_path}/{file_name}.jpg")
        label = self.annotation.iloc[idx]['class']

        if self.transform :

            image = self.transform(image)

        if self.target_transform :

            label = self.target_transform(label)

        return image , label

In [3]:
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])

In [4]:
train_annotation_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_train_datasets.csv'
train_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/train/images'

In [8]:
train_dataset = DatasetForClassification(train_annotation_path , train_images_path , transform )
train_dataloader = DataLoader(train_dataset , batch_size = 32 , shuffle = True)

### Dinov2

In [10]:
from transformers import AutoImageProcessor, Dinov2ForImageClassification
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [11]:
!nvidia-smi

Tue May 28 11:05:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   43C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base-imagenet1k-1-layer")
model = Dinov2ForImageClassification.from_pretrained("facebook/dinov2-base-imagenet1k-1-layer")
model.classifier = torch.nn.Linear(model.classifier.in_features, 1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/58.3k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/353M [00:00<?, ?B/s]

In [14]:
model = model.to(device)

In [23]:
import torch.optim as optim

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

def train_model(model, train_loader, criterion, optimizer, num_epochs = 3):

    model.train()

    for epoch in range(num_epochs):

        print(epoch)

        running_loss = 0.0

        for images, labels in tqdm(train_dataloader):

            labels = labels.to(device)
            labels = labels.float()

            inputs = image_processor(images, return_tensors="pt").to(device)

            optimizer.zero_grad()

            outputs = model(**inputs).logits
            loss = criterion(outputs.squeeze(), labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

        model.save_pretrained(f"/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/model_checkpoints/dinov2_{epoch}", from_pt=True)

    print("Training complete")

In [ ]:
# Train the model
train_model(model, train_dataloader, criterion, optimizer, num_epochs = 3)

0


  1%|          | 4/452 [01:37<2:59:58, 24.10s/it]

In [ ]:
def evaluate_model(model, val_loader):

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():

        for inputs, labels in val_loader:

            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)  # Reshape labels for binary classification

            outputs = model(inputs).logits
            predicted = torch.sigmoid(outputs).round()  # Apply sigmoid and round to get binary outputs
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy}%')

# Evaluate the model
evaluate_model(model, val_loader)

## YOLOv10 For Object Detection

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda --version

conda 23.11.0


In [ ]:
!git clone https://github.com/THU-MIG/yolov10.git
%cd yolov10

fatal: destination path 'yolov10' already exists and is not an empty directory.
/content/yolov10


In [ ]:
!conda create -n yolov10 python=3.10.2
!conda activate yolov10
!pip install -r requirements.txt
!pip install -e .

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/yolov10

  added / updated specs:
    - python=3.10.2


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-hd590300_5 
  ca-certificates    conda-forge/linux-64::ca-certificates-2024.2.2-hbcca054_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.40-hf3520f5_1 
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5 
  libgcc-ng          conda-forge/linux-64::libgcc-ng-13.2.0-h77fa898_7 
  libgomp            conda-forge/linux-64::libgomp-13.2.0-h77

In [ ]:
data_yaml = """
train: /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/train
val: /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/validation
test: /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/test
nc: 7
names: ['FCC' , 'FSS' , 'HCC' ,'cyst' , 'hemangioma' , 'dysplastic' , 'CCA']
"""

In [ ]:
data_yaml_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets'

with open (f'{data_yaml_path}/data.yaml' , 'w') as file :

    file.write(data_yaml)

## facebook/detr-resnet-50 For Object Detection

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests
import numpy as np

In [ ]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
image

In [ ]:
np.array(image).shape

In [ ]:
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

In [ ]:
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):

    box = [round(i, 2) for i in box.tolist()]

    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

In [ ]:
!pip install timm

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("object-detection", model="facebook/detr-resnet-50")

pipe(image)